# ***DISTRIBUCIÓN DE SEGMENTOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# *Data import*

In [2]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

### *Creación de los df por cada archivo json*

In [3]:
dataframes = {}
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

In [4]:
# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres, solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_sep_2023_pachuca', 'df_enero_2024_querertaro', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2023_pachuca', 'df_grupos_edad_pachuca', 'df_grupos_edad_queretaro', 'df_junio_2023_queretaro', 'df_salarios_pachuca', 'df_usuarios']


_________
# *Distribución*
### Selección de datos

In [5]:
# Crea los DataFrames para cada archivo con las columnas necesarias
segment_df_alfa_q_oct_2022_pachuca = df_alfa_q_oct_2022_pachuca[['id','categoria']]
segment_df_alfa_q_nov_2022_pachuca = df_alfa_q_nov_2022_pachuca[['id','categoria']]
segment_df_alfa_q_feb_2023_pachuca = df_alfa_q_feb_2023_pachuca[['id','categoria']]
segment_df_alfa_q_mar_2023_pachuca = df_alfa_q_mar_2023_pachuca[['id','categoria']]
segment_df_alfa_q_jun_2023_pachuca = df_alfa_q_jun_2023_pachuca[['id','categoria']]
segment_df_alfa_q_jul_2023_pachuca = df_alfa_q_jul_2023_pachuca[['id','categoria']]
segment_df_alfa_q_sep_2023_pachuca = df_alfa_q_sep_2023_pachuca[['id','categoria']] # Vacía

### Conteo único de datos "categoria"

In [6]:
# Lista de DataFrames
dataframes_list = [
    segment_df_alfa_q_oct_2022_pachuca,
    segment_df_alfa_q_nov_2022_pachuca,
    segment_df_alfa_q_feb_2023_pachuca,
    segment_df_alfa_q_mar_2023_pachuca,
    segment_df_alfa_q_jun_2023_pachuca,
    segment_df_alfa_q_jul_2023_pachuca,
    segment_df_alfa_q_sep_2023_pachuca
]

conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S,2257
1,D1,917
2,C3,747
3,B1,826
4,B2,755
5,B3,670
6,D2,562
7,C2,613
8,D3,438
9,A3,479


In [7]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,2257,S,1
1,917,D,1
2,747,C,3
3,826,B,1
4,755,B,2
5,670,B,3
6,562,D,2
7,613,C,2
8,438,D,3
9,479,A,3


In [8]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_dfs.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1         435
          2         450
          3         479
B         1         826
          2         755
          3         670
C         1         425
          2         613
          3         747
D         1         917
          2         562
          3         438
E         1         287
          2          47
          3           0
S         1        2257
Name: conteo_total, dtype: int64

### Totales de registros por segmento

In [9]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  9908


,segmento,total_por_segmento,porcentaje_del_total
0,A,1364,13.766653
1,B,2251,22.719015
2,C,1785,18.015745
3,D,1917,19.348002
4,E,334,3.371013
5,S,2257,22.779572


### Porcentaje de clase por segmento

In [10]:
#Obtener el total de registros por segmento y clase
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

,segmento,clase,porcentaje
0,S,1,100.000000
1,D,1,47.835159
2,D,2,29.316641
3,D,3,22.848200
4,C,3,41.848739
5,C,2,34.341737
6,C,1,23.809524
7,B,1,36.694802
8,B,2,33.540649
9,B,3,29.764549


### Porcentaje de cada clase en relación al % que representa su segmento del total.

In [11]:
temp_dfs = [] # Almacenar los DataFrames temporales de cada segmento
# Realizamos un bucle sobre cada segmento
for segmento in resultados['segmento'].unique():
    # Filtramos las clases correspondientes al segmento actual
    clases_segmento = segmentos_percent[segmentos_percent['segmento'] == segmento]
    
    # Calculamos el total de porcentaje del segmento actual
    total_porcentaje_segmento = resultados[resultados['segmento'] == segmento]['porcentaje_del_total'].iloc[0]
    
    # Calculamos el porcentaje que representa cada clase en relación al porcentaje del segmento
    porcentaje_clase = (clases_segmento['porcentaje'] / 100) * total_porcentaje_segmento
    
    # DataFrame temporal para el segmento actual
    temp_df = pd.DataFrame({
        'segmento': [segmento] * len(clases_segmento),
        'clase': clases_segmento['clase'].values,
        'porcentaje_clase': porcentaje_clase#.round()
    })
    
    # Agrega DataFrame temporal a la lista
    temp_dfs.append(temp_df)

# Concatena todos los DataFrames temporales en un solo DataFrame
porcentajes_clase = pd.concat(temp_dfs, ignore_index=True)
porcentajes_clase

,segmento,clase,porcentaje_clase
0,A,3,4.834477
1,A,1,4.390392
2,A,2,4.541784
3,B,1,8.336698
4,B,2,7.620105
5,B,3,6.762212
6,C,3,7.539362
7,C,2,6.186920
8,C,1,4.289463
9,D,1,9.255147


In [12]:
porcentajes_clase_ordenado = porcentajes_clase.pivot(index='segmento', columns='clase', values='porcentaje_clase')
porcentajes_clase_ordenado.fillna(0, inplace=True)
porcentajes_clase_ordenado

clase,1,2,3
segmento,,,
A,4.390392,4.541784,4.834477
B,8.336698,7.620105,6.762212
C,4.289463,6.186920,7.539362
D,9.255147,5.672184,4.420670
E,2.896649,0.474364,0.000000
S,22.779572,0.000000,0.000000


In [15]:
segmentos = porcentajes_clase_ordenado.index # Segmentos como etiquetas de eje x
clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos
# Definir los colores por clase
colores = {
    '1': '#3d45c0',
    '2': '#ff4853',
    '3': '#ffb038'
}
# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        x=segmentos,  # Eje x: Segmentos
        y=porcentajes_clase_ordenado[clase],  # Eje y: Porcentajes de clase
        name=f'Clase {clase}',  # Nombre de la serie de datos
        text=porcentajes_clase_ordenado[clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    title='Distribución de segmentos',
    xaxis=dict(title='Segmento'),
    yaxis=dict(
        title='Porcentaje',
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),
    barmode='stack',  # Apilar las barras
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_distsegmentos_v_pachuca', carpeta='graficas')

fig.show()

In [18]:
# Mantener orden en barras
segmentos = list(porcentajes_clase_ordenado.index)
segmentos_sorted = ['E','D','C','B','A','S']  # Orden deseado
segmentos_sorted.extend([segmento for segmento in segmentos if segmento not in segmentos_sorted])  # Agregar los segmentos restantes

clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos

# Definir los colores por clase
colores = {
    '1': '#3d45c0',
    '2': '#ff4853',
    '3': '#ffb038'
}

# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        y=segmentos_sorted,  # Eje y: Segmentos
        x=porcentajes_clase_ordenado.loc[segmentos_sorted, clase],  # Eje x: Porcentajes de clase
        name=f'{clase}',  # Nombre de la serie de datos
        orientation='h',  # Orientación horizontal
        text=porcentajes_clase_ordenado.loc[segmentos_sorted, clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]   # Color de las barras
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    title='Distribución de segmentos',
    yaxis=dict(
        title='Segmento', # Título del eje y
        #gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        #gridwidth=1,
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico      
    xaxis=dict(
        title='Porcentaje', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),  
    barmode='stack'  # Apilar las barras
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_distsegmentos_pachuca', carpeta='graficas')

fig.show()

_______________________________
___________
________

In [23]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668','#67c2df','#6773df']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, hole=0.5, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    title='Porcentaje de cada segmento' # Agregar título
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_dona_segmentosporc_pachuca', carpeta='graficas')

fig.show()